In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import datetime as dt
from datetime import timedelta, date

# Reflect Tables into SQLAlchemy ORM

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [4]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [5]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [6]:
# View all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [7]:
# Save references to each table
measurement = Base.classes.measurement
station = Base.classes.station

In [8]:
# Create our session (link) from Python to the DB
session = Session(engine)

##  Explore Database

In [9]:
## View table details
Base.metadata.tables

immutabledict({'measurement': Table('measurement', MetaData(bind=None), Column('id', INTEGER(), table=<measurement>, primary_key=True, nullable=False), Column('station', TEXT(), table=<measurement>), Column('date', TEXT(), table=<measurement>), Column('prcp', FLOAT(), table=<measurement>), Column('tobs', FLOAT(), table=<measurement>), schema=None), 'station': Table('station', MetaData(bind=None), Column('id', INTEGER(), table=<station>, primary_key=True, nullable=False), Column('station', TEXT(), table=<station>), Column('name', TEXT(), table=<station>), Column('latitude', FLOAT(), table=<station>), Column('longitude', FLOAT(), table=<station>), Column('elevation', FLOAT(), table=<station>), schema=None)})

In [10]:
# get the table names (same result as: Base.classes.keys() )
inspector = inspect(engine)
inspector.get_table_names()

['measurement', 'station']

In [11]:
# for first table, get a list of column names and types in a more pleasing format
columns = inspector.get_columns('measurement')
for c in columns:
    print(c['name'], c['type'])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [12]:
# for second table, get a list of column names and types in a more pleasing format
columns = inspector.get_columns('station')
for c in columns:
    print(c['name'], c['type'])

id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


In [13]:
# for the first table, view the data structure
engine.execute('select * from measurement limit 5').fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-06', None, 73.0)]

In [14]:
engine.execute('select * from station limit 5').fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]

# Exploratory Precipitation Analysis

In [47]:
# Find the most recent date in the data set.
# recent_date_str = session.query(measurement.date).order_by(measurement.date.desc()).first()
# recent_date_str


('2017-08-23')

In [50]:
# Find the most recent date in the data set.
recent_date_str = session.query(func.max(measurement.date)).one()[0]
recent_date_str

'2017-08-23'

In [70]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database.
for recent_date in recent_date_str:    
    recent_date = dt.datetime.strptime(recent_date_str, "%Y-%m-%d")
recent_date

# Calculate the date one year from the last date in data set.
one_year_ago = recent_date - timedelta(days=365)
one_year_ago

# Perform a query to retrieve the data and precipitation scores
year_precipitations_query = session.query(measurement.date, measurement.prcp)
year_precipitations_query

## Filter and extract the data to include only the past year's of precipitations
past_year_precipitations_query = session.query(measurement.date, measurement.prcp).filter(
                                and_(
                                (measurement.date >= one_year_ago).all()
                                (measurement.precipitation)
        )
)
past_year_precipitations_query


# Save the query results as a Pandas DataFrame and set the index to the date column
past_year_precipitations_df = pd.DataFrame(past_year_precipitations_query, columns=['date','precipitation'])
past_year_precipitations_df.set_index(past_year_precipitations_df['date'], inplace=True)

# Sort the dataframe by date
past_year_precipitations_df.sort_index()
past_year_precipitations_df



,date,precipitation
date,,
2016-08-24,2016-08-24,0.08
2016-08-25,2016-08-25,0.08
2016-08-26,2016-08-26,0.00
2016-08-27,2016-08-27,0.00
2016-08-28,2016-08-28,0.01
...,...,...
2017-08-19,2017-08-19,0.09
2017-08-20,2017-08-20,NaN
2017-08-21,2017-08-21,0.56


In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
# ptp_rows = [{"Date": result[1], "PTP": result[0]} for result in results]
# ptp_rows

# pd.DataFrame(ptp_rows).set_index("Date").boxplot(patch_artist=True)
# plt.title("IBM PTPs")
# plt.show()

In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data


# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset


In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.


In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [ ]:
# Close Session
session.close()